In [ ]:
#!/usr/bin/env python3
"""
age_prediction.py

Script đơn giản để dự đoán tuổi (số ngày) của cây lúa từ ảnh đầu vào,
sử dụng mô hình hồi quy đã huấn luyện sẵn.

Usage:
    python age_prediction.py \
        --model-path PATH_TO_MODEL.pt \
        --image-path PATH_TO_IMAGE.jpg \
        [--model-type {baseline,resnet}] \
        [--no-cuda]

Dependencies:
    - torch>=2.6.0
    - torchvision>=0.15.0
    - pillow
"""

import argparse
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision.models import resnet18
from PIL import Image

# -------------------------------------------------------------------
# 1. Định nghĩa các kiến trúc mô hình
# -------------------------------------------------------------------

class Baseline(nn.Module):
    """Mô hình CNN cơ bản cho hồi quy tuổi."""
    def __init__(self):
        super(Baseline, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout2d(0.1),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout2d(0.1),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout2d(0.2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        self.regressor = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        x = self.features(x)
        return self.regressor(x)


class ResNetAge(nn.Module):
    """Mô hình ResNet18 gốc cho hồi quy tuổi."""
    def __init__(self):
        super(ResNetAge, self).__init__()
        base = resnet18(weights=None)
        # Loại bỏ lớp fc cuối cùng
        self.feature_extractor = nn.Sequential(*list(base.children())[:-1])
        self.regressor = nn.Sequential(
            nn.Flatten(),
            nn.Linear(base.fc.in_features, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        x = self.feature_extractor(x)  # Output shape: (B, C, 1, 1)
        return self.regressor(x)


# -------------------------------------------------------------------
# 2. Định nghĩa transform cho ảnh
# -------------------------------------------------------------------

def get_transform():
    """Chuỗi transform resize, tensorize và normalize ảnh."""
    return transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])


# -------------------------------------------------------------------
# 3. Hàm dự đoán tuổi
# -------------------------------------------------------------------

def predict_age(image_path: str, model: nn.Module, device: torch.device) -> float:
    """
    Dự đoán tuổi từ ảnh.

    Args:
        image_path: đường dẫn đến file ảnh.
        model: mô hình đã load state_dict.
        device: torch.device('cpu' hoặc 'cuda').

    Returns:
        Tuổi (số ngày) dạng float.
    """
    img = Image.open(image_path).convert('RGB')
    tensor = get_transform()(img).unsqueeze(0).to(device)  # shape: (1,3,224,224)
    model.eval()
    with torch.no_grad():
        output = model(tensor).item()
    return output


# -------------------------------------------------------------------
# 4. Hàm main để parse args và chạy
# -------------------------------------------------------------------

def main():
    parser = argparse.ArgumentParser(
        description='Predict age (days) from an image using a trained model'
    )
    parser.add_argument(
        '--model-path', required=True,
        help='Đường dẫn tới file mô hình (.pt)'
    )
    parser.add_argument(
        '--image-path', required=True,
        help='Đường dẫn tới file ảnh cần dự đoán'
    )
    parser.add_argument(
        '--model-type', choices=['baseline','resnet'], default='resnet',
        help='Chọn kiến trúc mô hình: baseline hoặc resnet'
    )
    parser.add_argument(
        '--no-cuda', action='store_true',
        help='Thêm flag này để dùng CPU ngay cả khi có GPU'
    )
    args = parser.parse_args()

    # Thiết lập device
    use_cuda = torch.cuda.is_available() and not args.no_cuda
    device = torch.device('cuda' if use_cuda else 'cpu')

    # Khởi tạo mô hình
    if args.model_type == 'baseline':
        model = Baseline().to(device)
    else:
        model = ResNetAge().to(device)

    # Load trạng thái mô hình
    state = torch.load(args.model_path, map_location=device)
    model.load_state_dict(state)

    # Dự đoán và in kết quả
    age_days = predict_age(args.image_path, model, device)
    print(f'Predicted age: {age_days:.2f} days')


if __name__ == '__main__':
    main()
